In [ ]:
import os
import sys
import re
import math

from matplotlib import pyplot as plt
import numpy as np
#import pandas as pd
#import seaborn as sns

%matplotlib inline

r = re.compile(r"Batch (\d+), prob = ([^\s]+)")


In [ ]:
def _get_log_extension():
    return '.stderr'

In [ ]:
def parse_log(log_file):
    probs = []

    with open(log_file) as f:
        last_class = None

        for l in f.readlines():
            matches = r.search(l)

            if matches:
                batch = int(matches.groups()[0])

                if last_class != batch:
                    last_class = batch
                    probs.append([])

                probs[last_class].append(float(matches.groups()[1]))

    # row x col = alignments x probabilities
    return np.asarray(probs)


In [ ]:
input_folder = '/media/schenock/01D1B41ED893D2C0/Projects/LOUD_OUT_FILES'
threshold = 0.9

In [ ]:
log_files = sorted([i for i in os.listdir(input_folder) if i.endswith(_get_log_extension())])

cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

In [ ]:
dfs = []
for log_file in log_files:
    print("Parsing:", log_file)
    m = parse_log(os.path.join(input_folder, log_file))
    diag = m.diagonal()
    y_pred = np.argmax(diag)

    print('\tIndex:', y_pred)
    print('\tClass:', y_pred + 1)
    print('Probability:', diag[y_pred])
    #print('Diagonal: ', diag)

    sums = np.apply_along_axis(np.sum, 0, m)
    ratios = diag / sums
    ratios_th = ratios.copy()
    ratios_th[diag < threshold] = 0
    y_pred = np.argmax(ratios_th)

    plt.figure()
    plt.title(log_file)
    plt.imshow(m, interpolation='none', cmap='Blues')

    plt.figure()
    plt.subplot(511)
    plt.title("diag")
    plt.imshow(diag.reshape((1, 43)), interpolation='none', cmap='Blues')

    plt.subplot(512)
    plt.title("sums")
    plt.imshow(sums.reshape((1, 43)), interpolation='none', cmap='Blues')

    plt.subplot(513)
    plt.title("ratios")
    plt.imshow(ratios.reshape((1, 43)), interpolation='none', cmap='Blues')

    plt.subplot(514)
    plt.title("ratios_th")
    plt.imshow(ratios_th.reshape((1, 43)),
               interpolation='none', cmap='Blues')
    
#     df = pd.DataFrame(m)
#     dfs.append(df)
